# 코랩 설정

아래 순서에 따라 코랩 환경을 설정합니다. 
1. 메뉴 > 런타임 > 런타임 유형 변경에서 "GPU"를 선택합니다.

아래 명령으로 data 폴더를 생성합니다.

In [ ]:
!mkdir data

# 데이터셋 가져오기

아래 코드를 통해 데이터셋을 다운로드 받습니다.

In [ ]:
!curl -d '[{"originFilename":"qm9_train_data.pt","convertFilename":"20220919004741_QbYk.pt"}]' -H "Content-Type: application/json" -X POST https://cdn.aifactory.space/download/additionalFiles -o ./data/qm9_train_data.pt
!curl -d '[{"originFilename":"qm9_test_data.pt","convertFilename":"20220919004756_XrdQ.pt"}]' -H "Content-Type: application/json" -X POST https://cdn.aifactory.space/download/additionalFiles -o ./data/qm9_test_data.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 93.7M  100 93.7M  100    83  28.3M     25  0:00:03  0:00:03 --:--:-- 28.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30.8M  100 30.8M  100    82  19.5M     51  0:00:01  0:00:01 --:--:-- 19.5M


# 베이스라인 코드 구동에 필요한 파일다운로드

In [ ]:
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/gnn.yaml
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/qm9_dataset.py
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/model.py

--2022-09-20 08:21:34--  https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/gnn.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 454 [text/plain]
Saving to: ‘gnn.yaml’

gnn.yaml            100%[===================>]     454  --.-KB/s    in 0s      

2022-09-20 08:21:34 (32.1 MB/s) - ‘gnn.yaml’ saved [454/454]

--2022-09-20 08:21:34--  https://raw.githubusercontent.com/aifactory-team/AFCompetition/master/2106/qm9_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7680 (7.5K) [text/plain]
Saving t

# 베이스라인 구동에 필요한 패키지 설치

dgl 패키지를 설치하기 위해 현재 코랩의 파이썬 버전 및 cuda 버전을 확인합니다.

In [ ]:
import sys
print(sys.version)

import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

3.7.14 (default, Sep  8 2022, 00:06:44) 
[GCC 7.5.0]
Torch version:1.12.1+cu113
cuda version: 11.3
cudnn version:8302


https://www.dgl.ai/pages/start.html 에 접속해서 파이썬 및 코랩 버전에 해당하는 설치 명령을 복사합니다.

In [ ]:
!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 239.1 MB 18 kB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 281 kB 61.7 MB/s 
     |████████████████████████████████| 103 kB 51.7 MB/s 
     |████████████████████████████████| 109 kB 64.6 MB/s 
     |████████████████████████████████| 78 kB 7.9 MB/s 
     |████████████████████████████████| 36.8 MB 39 kB/s 
     |████████████████████████████████| 51 kB 694 kB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 41 kB 482 kB/s 
     |████████████████████████████████| 3.2 MB 54.5 MB/s 
     |████████████████████████████████| 546 kB 68.0 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 121 kB 76.2 MB/s 
     |████████████████████████████████| 90 kB 1

In [ ]:
!pip install omegaconf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 베이스라인 코드 실행 및 제출 결과 파일 생성

In [ ]:
import os
import torch
import numpy as np
import argparse
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader

from qm9_dataset import QM9DGLDataset
from omegaconf import OmegaConf
from model import GNN_model
import torch.nn as nn
from tqdm import tqdm

"""
    GPU Setup
"""
def gpu_setup(use_gpu, gpu_id):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

    if torch.cuda.is_available() and use_gpu:
        print('cuda available with GPU:',torch.cuda.get_device_name(0))
        device = torch.device("cuda")
    else:
        print('cuda not available')
        device = torch.device("cpu")
    return device


def to_np(x):
    return x.cpu().detach().numpy()


def train_epoch(epoch, model, loss_fnc, dataloader, optimizer, scheduler, FLAGS, device):
    model.train()
    num_iters = len(dataloader)
    for i, (g, y) in enumerate(dataloader):
        g = g.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        # run model forward and compute loss
        pred = model(g)
        loss = loss_fnc(pred, y)

        # backprop
        loss.backward()
        optimizer.step()

        if i % FLAGS.train_params.print_epoch_interval == 0:
            print(f"[{epoch}|{i}] loss: {loss.item():.5f}")

        scheduler.step(epoch + i / num_iters)


def val_epoch(epoch, model, loss_fnc, dataloader, FLAGS, device):
    model.eval()
    total_loss = 0
    rescale_loss = 0
    for i, (g, y) in enumerate(tqdm(dataloader)):
        g = g.to(device)
        y = y.to(device)

        # run model forward and compute loss
        pred = model(g)
        loss = loss_fnc(pred, y)

        total_loss += loss.item()

    print(f"...[{epoch}|val] loss: {total_loss:.5f}")


def run_test(model, dataloader, device):
    model.eval()
    preds = []
    for g in tqdm(dataloader):
        g = g.to(device)
        pred = model(g)
        preds.append(to_np(pred))

    return np.concatenate(preds, axis=0)

# Loss function
def l1_loss(pred, target):
    loss = F.l1_loss(pred, target)
    return loss

################ 1. parsing arguments

parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='./gnn.yaml', help='configration for model')
parser.add_argument('--pretrained_path', type=str, default=None, help='configration for model')
args, extra_args = parser.parse_known_args()
FLAGS = OmegaConf.load(args.config)

# Create model directory
if not os.path.isdir(FLAGS.out_dir):
    os.makedirs(FLAGS.out_dir)

# Fix SEED
torch.manual_seed(FLAGS.train_params.seed)
np.random.seed(FLAGS.train_params.seed)

# Automatically choose GPU if available
device = gpu_setup(FLAGS['gpu']['use'], FLAGS['gpu']['id'])



################ 2. Prepare data
dataset = QM9DGLDataset(FLAGS.data_path,
                              FLAGS.task,
                              file_name='qm9_train_data.pt',
                              mode='train')

train_dataset, val_dataset = dataset.train_val_random_split(0.8)


train_loader = DataLoader(train_dataset,
                          batch_size=FLAGS.train_params.batch_size,
                          shuffle=True,
                          collate_fn=dataset.collate_fn,
                          num_workers=FLAGS.data.num_workers)

val_loader = DataLoader(val_dataset,
                        batch_size=FLAGS.train_params.batch_size,
                        shuffle=False,
                        collate_fn=dataset.collate_fn,
                        num_workers=FLAGS.data.num_workers)

# Test Dataset
test_dataset = QM9DGLDataset(FLAGS.data_path,
                         FLAGS.task,
                         file_name='qm9_test_data.pt',
                         mode='test')

test_loader = DataLoader(test_dataset,
                         batch_size=FLAGS.train_params.batch_size,
                         shuffle=False,
                         collate_fn=test_dataset.collate_fn,
                         num_workers=FLAGS.data.num_workers)

FLAGS.train_size = len(train_dataset)
FLAGS.val_size = len(val_dataset)
FLAGS.test_size = len(test_dataset)
print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

################ 2. Prepare model
model = GNN_model(FLAGS.graph_encoder_params)

if args.pretrained_path is not None:
    model.load_state_dict(torch.load(args.pretrained_path))

model.to(device)

criterion = l1_loss
optimizer = optim.Adam(model.parameters(), lr=FLAGS.train_params.init_lr)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
                                                           FLAGS.train_params.epochs,
                                                           eta_min=FLAGS.train_params.min_lr)
                                                 

################ 2. Start training

# Run training
print('Begin training')
for epoch in range(FLAGS.train_params.epochs):
    train_epoch(epoch, model, criterion, train_loader, optimizer, scheduler, FLAGS, device)
    val_epoch(epoch, model, criterion, val_loader, FLAGS, device)
    
    # save checkpoint
    save_path = os.path.join(FLAGS.out_dir, f"{FLAGS.model}_{FLAGS.task}_{epoch}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Saved checkpoint: {save_path}")


################ 3. Test
print('Begin test')
predictions = run_test(model, test_loader, device)
np.savetxt('pred.csv', predictions)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
cuda available with GPU: Tesla T4
Loaded train-set, task: mu, source: ./data/, length: 98123
Loaded test-set, task: mu, source: ./data/, length: 32708
Train set size: 78498
Validation set size: 19625
Test set size: 32708
Begin training
[0|0] loss: 2.65366
[0|5] loss: 2.18377
[0|10] loss: 1.65425
[0|15] loss: 1.22601
[0|20] loss: 1.46055
[0|25] loss: 1.19969
[0|30] loss: 1.13633
[0|35] loss: 1.32136
[0|40] loss: 1.21862
[0|45] loss: 1.06600
[0|50] loss: 1.17025
[0|55] loss: 1.21068
[0|60] loss: 1.13126
[0|65] loss: 1.15721
[0|70] loss: 1.04274
[0|75] loss: 1.05481
[0|80] loss: 0.93207
[0|85] loss: 1.09956
[0|90] loss: 1.08441
[0|95] loss: 1.05560
[0|100] loss: 1.09095
[0|105] loss: 1.06502
[0|110] loss: 1.08098
[0|115] loss: 1.05590
[0|120] loss: 1.03363
[0|125] loss: 1.15154
[0|130] loss:

100%|██████████| 154/154 [00:09<00:00, 15.59it/s]


...[0|val] loss: 148.51277
Saved checkpoint: ./results/GNN_model_mu_0.pt
Begin test


100%|██████████| 256/256 [00:15<00:00, 16.37it/s]


# 참고사항

현재 설정한 모든 Epoch의 학습을 완료하면 이때까지 fitting된 모델 Weight로 테스트 셋에 대한 예측을 하게 됩니다. 따라서, Validation 결과상 Best 모델을 선택 해 추론을 하고 싶다면, 추가로 코드를 구현하셔야 합니다.

# 결과 파일 제출

1. 코랩 파일탭에서 생성된 "pred.csv"을 다운로드 합니다.
1. https://aifactory.space/competition/submission/2106 에 접속합니다.
1. "pred.csv"파일을 위 페이지에서 제출합니다.
1. https://aifactory.space/competition/leaderboard/2106 리더보드에서 자신의 점수 및 순위를 확인합니다.